In [1]:
from binance import Client, ThreadedWebsocketManager, ThreadedDepthCacheManager
import pandas as pd
import numpy as np
import os
from binance import Client, ThreadedWebsocketManager, ThreadedDepthCacheManager
import pandas as pd
import numpy as np
import json
import datetime
import time

# Run Once

In [14]:
%store -r  bruno_api_key
%store -r bruno_secret_key

In [10]:
# Binance API keys
api_key = bruno_api_key
secret_key = bruno_secret_key
# Initialize Binance client
client = Client(api_key, secret_key)

# List of top_50 currencies to fetch data for
top_50 = ['BTCUSDT', 'ETHUSDT','BNBUSDT','XRPUSDT','BUSDUSDT','ADAUSDT',
          'DOGEUSDT','MATICUSDT','SOLUSDT','DOTUSDT','SHIBUSDT','LTCUSDT',
          'TRXUSDT','AVAXUSDT','UNIUSDT','ATOMUSDT','LINKUSDT','ETCUSDT','XMRUSDT',
          'BCHUSDT','HBARUSDT','XLMUSDT','APTUSDT','FILUSDT','LDOUSDT','NEARUSDT','APEUSDT',
          'ALGOUSDT','VETUSDT','QNTUSDT','ICPUSDT','GRTUSDT','MANAUSDT','FTMUSDT','EOSUSDT',
          'AAVEUSDT','SANDUSDT','EGLDUSDT','FLOWUSDT','THETAUSDT','AXSUSDT','XTZUSDT','LUNCUSDT',
          'CHZUSDT','MINAUSDT','IMXUSDT','FXSUSDT','ZECUSDT','CAKEUSDT','MKRUSDT']

# Get historical klines data for top_50 from 2022 until yesterday
interval = Client.KLINE_INTERVAL_1DAY
start_date = datetime.datetime(2022, 1, 1)
end_date = datetime.datetime.today() - datetime.timedelta(days=1)
end_date_str = end_date.strftime('%d %b %Y')
historical_data = {}
for symbol in top_50:
    klines = client.get_historical_klines(symbol, interval, start_date.strftime('%d %b %Y'), '16 Jul 2023')
    df = pd.DataFrame(klines, columns=['open_time', 'open', 'high', 'low', 'close', 'volume', 'close_time',
                                           'quote_asset_volume', 'number_of_trades', 'tb_base_volume',
                                           'tb_quote_volume', 'ignore'])
    df['symbol'] = symbol
    historical_data[symbol] = df
#historical[i] = client.get_historical_klines(i, Client.KLINE_INTERVAL_1DAY, '1 Jan 2022', yesterday)

# Concatenate historical data into a single dataframe
df = pd.concat(historical_data.values(), ignore_index=True)

# Convert timestamp to datetime
df['open_time'] = pd.to_datetime(df['open_time'], unit='ms')
df['close_time'] = pd.to_datetime(df['close_time'], unit='ms')

#convert string columns into numbers 
numeric_columns = ['open', 'high', 'low', 'close','volume','quote_asset_volume','tb_base_volume','tb_quote_volume']
df[numeric_columns] = df[numeric_columns].apply(pd.to_numeric, errors='coerce')

# day of week column and day of month
df['day_of_week'] = df['close_time'].dt.dayofweek
df['day_of_month'] = df['close_time'].dt.day

# Pct change column
df['price_pct_change'] = df.groupby('symbol')['close'].pct_change().round(3)

# Create column named 'price_buckets'
df['price_buckets'] = df.groupby('symbol')['close'].transform(lambda x: pd.cut(x, 8))
# Convert price buckets to string
df['price_buckets'] = df['price_buckets'].astype(str)
# Remove characters and replace the comma
df['price_buckets'] = df['price_buckets'].str.replace("'", "").str.replace("[","").str.replace("(","").str.replace(")","").str.replace("]","").str.replace(",", " -")

#Save historical data to file
df.to_csv('historical_data.csv', index=False)

/var/folders/cg/vf981v4x5tz8b5szmpt19_xm0000gp/T/ipykernel_25350/317025203.py:54: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['price_buckets'] = df['price_buckets'].str.replace("'", "").str.replace("[","").str.replace("(","").str.replace(")","").str.replace("]","").str.replace(",", " -")


# Schedule Run

In [17]:
from binance import Client, ThreadedWebsocketManager, ThreadedDepthCacheManager
import pandas as pd
import numpy as np
import os
from binance import Client, ThreadedWebsocketManager, ThreadedDepthCacheManager
import pandas as pd
import numpy as np
import json
import datetime
import time

historical_data_file = 'historical_data.csv'
# Binance API keys
api_key = bruno_api_key
secret_key = bruno_secret_key
# Initialize Binance client
client = Client(api_key, secret_key)

# List of top_50 currencies to fetch data for
top_50 = ['BTCUSDT', 'ETHUSDT','BNBUSDT','XRPUSDT','BUSDUSDT','ADAUSDT',
          'DOGEUSDT','MATICUSDT','SOLUSDT','DOTUSDT','SHIBUSDT','LTCUSDT',
          'TRXUSDT','AVAXUSDT','UNIUSDT','ATOMUSDT','LINKUSDT','ETCUSDT','XMRUSDT',
          'BCHUSDT','HBARUSDT','XLMUSDT','APTUSDT','FILUSDT','LDOUSDT','NEARUSDT','APEUSDT',
          'ALGOUSDT','VETUSDT','QNTUSDT','ICPUSDT','GRTUSDT','MANAUSDT','FTMUSDT','EOSUSDT',
          'AAVEUSDT','SANDUSDT','EGLDUSDT','FLOWUSDT','THETAUSDT','AXSUSDT','XTZUSDT','LUNCUSDT',
          'CHZUSDT','MINAUSDT','IMXUSDT','FXSUSDT','ZECUSDT','CAKEUSDT','MKRUSDT']

#load historical data
historical_data = pd.read_csv(historical_data_file, parse_dates=['close_time'])
last_date = historical_data['close_time'].max().date()

# Start fetching new data
fetch_start_date = last_date + datetime.timedelta(days=1)

# Calculate the date to end fetching new data
fetch_end_date = datetime.datetime.today().date() - datetime.timedelta(days=1)

# setting up interval for fetching
interval = Client.KLINE_INTERVAL_1DAY
# Fetch historical data from Binance API for missing period
if fetch_start_date <= fetch_end_date:
    new_data = {}
    for symbol in top_50:
        klines = client.get_historical_klines(symbol, interval, fetch_start_date.strftime('%d %b %Y'),
                                              fetch_end_date.strftime('%d %b %Y'))
        df = pd.DataFrame(klines, columns=['open_time', 'open', 'high', 'low', 'close', 'volume', 'close_time',
                                           'quote_asset_volume', 'number_of_trades', 'tb_base_volume',
                                           'tb_quote_volume', 'ignore'])
        df['symbol'] = symbol
        new_data[symbol] = df

# Concatenate the values of new_data to a df
df = pd.concat(new_data.values(), ignore_index=True)

# Convert timestamp to datetime
df['open_time'] = pd.to_datetime(df['open_time'], unit='ms')
df['close_time'] = pd.to_datetime(df['close_time'], unit='ms')

#convert string columns into numbers 
numeric_columns = ['open', 'high', 'low', 'close','volume','quote_asset_volume','tb_base_volume','tb_quote_volume']
df[numeric_columns] = df[numeric_columns].apply(pd.to_numeric, errors='coerce')

# day of week column and day of month
df['day_of_week'] = df['close_time'].dt.dayofweek
df['day_of_month'] = df['close_time'].dt.day

# Append new data to historical data
# Changing the order due to the fact that we want to not have any NA values for price_pct change
# Combine the old and new data
combined_df = pd.concat([historical_data, df], ignore_index=True)

# Pct change column
combined_df['price_pct_change'] = combined_df.groupby('symbol')['close'].pct_change().round(3)
df['price_pct_change'] = combined_df['price_pct_change']

# Calculate the bin edges based on the 'close' prices in the combined data
bin_edges = pd.cut(combined_df['close'], bins=8).cat.categories

# Apply the calculated bin edges to the new data
df['price_buckets'] = pd.cut(df['close'], bins=bin_edges)

# Convert the 'price_buckets' column to string and remove unwanted characters
df['price_buckets'] = df['price_buckets'].astype(str)
df['price_buckets'] = df['price_buckets'].str.replace("'", "").str.replace("[","").str.replace("(","").str.replace(")","").str.replace("]","").str.replace(",", " -")

# add the new rows to the existing csv file
# Append new data to historical data
final_df = pd.concat([historical_data, df], ignore_index=True)
# Save historical data to file
final_df.to_csv(historical_data_file, index=False)

/var/folders/cg/vf981v4x5tz8b5szmpt19_xm0000gp/T/ipykernel_25350/4212496520.py:84: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['price_buckets'] = df['price_buckets'].str.replace("'", "").str.replace("[","").str.replace("(","").str.replace(")","").str.replace("]","").str.replace(",", " -")
